## Sample Data

In [1]:
data LinPlat;
  input x y mu;
datalines;
0 10.48 10
1 9.61 10.5
2 10.93 11
3 11.66 11.5
4 11.91 12
5 11.79 12.5
6 13.55 13
7 14 13.5
8 13.36 14
9 14.44 14.3
10 14.54 14.6
11 14.44 14.9
12 15.25 15
13 14.62 15
13 15.46 15
14 14.81 15
15 14.84 15
16 14.82 15
17 15.36 15
18 14.89 15
19 15.53 15
20 14.86 15
21 15.18 15
22 13.98 15
23 15.3 15
24 15.08 15
25 14.89 15
26 15.51 15
27 14.49 15
28 15.48 15
29 15.16 15
30 15.62 15
;
/* Figure 1 */
symbol1 value=' ' i=join color=blue;
symbol2 value=diamond i=none color=black;
proc gplot data=linplat;
 plot (mu y)*x/overlay;
 title1 "linear plataeu example";
 title2 "BLUE='truth'; BLACK diamonds = the observed data";
run;


SAS Connection established. Subprocess id is 244716



## NLIN

In [18]:

/* Mitscherlich model */
/* program 1 */
proc nlin data=linplat listall;
 parms a=15 b=0.33 c=0.25;
 model y=a*(1-b*exp(-c*x));
 output out=mtsch predicted=yhat;
run;


## NLMIXED

In [2]:

/* program 2 */
proc nlmixed data=linplat;
 parms a=15 b=0.33 c=0.25;
 eta=a*(1-b*exp(-c*x));
 model y~normal(eta,var);
 predict eta out=predmtsch;
symbol1 i=none value=diamond color=black;
symbol2 i=join value=' ' color=green;
symbol3 i=join value=' ' color=blue;
/*proc print data=predmtsch;*/
run;
proc gplot data=predmtsch;
 plot (y pred mu)*x/overlay vref=14.9 href=15;
 title1 "Mitscherlich model";
 title2 " ";
run; 

## IML (Pseudolikelihood)

In [3]:
proc print data=NLin_S5 (obs=5);
run;

In [21]:
* This is the Logistic code;
proc iml;
 use NLin_S5;
 read all;
 
 * Initial Values;
 a = 9.6;
 b = 30;
 c = -5;
 d = 0.5;
 
 ai_start = 0.1;
 bi_start = 0.1;
 ci_start = 0.1;
 di_start = 0.1;

 sigma_residual = 0.7;
 sigma_random = {0.6,2.5,0.15,0.005};
 
 * Begin Program;
 nobs = nrow(y);
 design = design(id);
 n_effects = ncol(design);
 n_sub = nobs/n_effects;
 crit = 1;
 niter = 0;
 print nobs n_effects n_sub;
 
 ai = j(n_effects,1,ai_start);
 bi = j(n_effects,1,bi_start);
 ci = j(n_effects,1,ci_start);
 di = j(n_effects,1,di_start); 
 
 beta_fixed = a//b//c//d;
 beta_random = ai//bi//ci//di;
 
 ai_x = ai@j(n_sub,1,1);
 bi_x = bi@j(n_sub,1,1);
 ci_x = ci@j(n_sub,1,1);
 di_x = di@j(n_sub,1,1);

 fa = j(nobs,1,1);
 fb = 1 / (1 + EXP(- (c + ci_x + (d + di_x) # x)));
 fc = - (- EXP(- (c + ci_x + (d + di_x) # x)) # (b + bi_x) / (1 + EXP(- (c + ci_x + (d + di_x) # x)))##2);
 fd = - (- x # EXP(- (c + ci_x + (d + di_x) # x)) # (b + bi_x) / (1 + EXP(- (c + ci_x + (d + di_x) # x)))##2);
 xstar = fa||fb||fc||fd;

 fai = design;
 fbi = design#fb;
 fci = design#fc;
 fdi = design#fd;
 zstar = fai||fbi||fci||fdi;
 
 yhat = a + ai_x + ((b + bi_x) / (1 + exp(- (c + ci_x + (d + di_x) # x))));
 ystar = y - yhat + xstar*beta_fixed + zstar*beta_random;
 
  sigma_ai = i(n_effects)#sigma_random[1];
  sigma_bi = i(n_effects)#sigma_random[2];
  sigma_ci = i(n_effects)#sigma_random[3];
  sigma_di = i(n_effects)#sigma_random[4];
  
  g_side = block(sigma_ai, sigma_bi, sigma_ci, sigma_di);
  g_inv = inv(g_side);
  
  r_side = i(nobs)#sigma_residual;
  r_inv = inv(r_side);
  
  
do while (crit>1e-7);
  lhs = ((xstar`*r_inv*xstar)||(xstar`*r_inv*zstar)) //
        ((zstar`*r_inv*xstar)||(zstar`*r_inv*zstar+g_inv));
  rhs = (xstar`*r_inv*ystar)//(zstar`*r_inv*ystar);
  solution = inv(lhs)*rhs;
  beta_fixed_new = solution[1:4];
  beta_random_new = solution[5:nrow(solution)];
  beta_random_matrix = (shape(beta_random_new,4,n_effects))`;

  a = beta_fixed_new[1];
  b = beta_fixed_new[2];
  c = beta_fixed_new[3];
  d = beta_fixed_new[4];
  
  ai = beta_random_matrix[,1];
  bi = beta_random_matrix[,2];
  ci = beta_random_matrix[,3];
  di = beta_random_matrix[,4];

  ai_x = ai@j(n_sub,1,1);
  bi_x = bi@j(n_sub,1,1);
  ci_x = ci@j(n_sub,1,1);
  di_x = di@j(n_sub,1,1);
  
  yhat = a + ai_x + ((b + bi_x) / (1 + exp(- (c + ci_x + (d + di_x) # x))));

  fa = j(nobs,1,1);
  fb = 1 / (1 + EXP(- (c + ci_x + (d + di_x) # x)));
  fc = - (- EXP(- (c + ci_x + (d + di_x) # x)) # (b + bi_x) / (1 + EXP(- (c + ci_x + (d + di_x) # x)))##2);
  fd = - (- x # EXP(- (c + ci_x + (d + di_x) # x)) # (b + bi_x) / (1 + EXP(- (c + ci_x + (d + di_x) # x)))##2);
  xstar = fa||fb||fc||fd;

  fai = design;
  fbi = design#fb;
  fci = design#fc;
  fdi = design#fd;
  zstar = fai||fbi||fci||fdi;
  
  ystar = y - yhat + xstar*beta_fixed_new + zstar*beta_random_new;
  
  sigma_ai = i(n_effects)#sigma_random[1];
  sigma_bi = i(n_effects)#sigma_random[2];
  sigma_ci = i(n_effects)#sigma_random[3];
  sigma_di = i(n_effects)#sigma_random[4];
  
  g_side = block(sigma_ai, sigma_bi, sigma_ci, sigma_di);
  g_inv = inv(g_side);
  
  r_side = i(nobs)#sigma_residual;
  r_inv = inv(r_side);

  var_fun = zstar*g_side*zstar`+r_side;
  var_inv = inv(var_fun);

  p = var_inv-var_inv*xstar*inv(xstar`*var_inv*xstar)*xstar`*var_inv;
  
  dv_ai = fai*fai`;
  dv_bi = fbi*fbi`;
  dv_ci = fci*fci`;
  dv_di = fdi*fdi`;
  dv_e = i(nobs); 
  
 /* score */
  sca = (-0.5)#trace(p*dv_ai) + 
          (1/2)#((ystar-xstar*beta_fixed_new)`*var_inv*dv_ai*var_inv*(ystar-xstar*beta_fixed_new));
  scb = (-0.5)#trace(p*dv_bi) + 
          (1/2)#((ystar-xstar*beta_fixed_new)`*var_inv*dv_bi*var_inv*(ystar-xstar*beta_fixed_new));
  scc = (-0.5)#trace(p*dv_ci) + 
          (1/2)#((ystar-xstar*beta_fixed_new)`*var_inv*dv_ci*var_inv*(ystar-xstar*beta_fixed_new));
  scd = (-0.5)#trace(p*dv_di) + 
          (1/2)#((ystar-xstar*beta_fixed_new)`*var_inv*dv_di*var_inv*(ystar-xstar*beta_fixed_new));
  sce = (-0.5)#trace(p*dv_e) + 
          (1/2)#((ystar-xstar*beta_fixed_new)`*var_inv*dv_e*var_inv*(ystar-xstar*beta_fixed_new));
  score = sca//scb//scc//scd//sce;

  /*information */
  h11=0.5#trace(p*dv_ai*p*dv_ai);
  h12=0.5#trace(p*dv_ai*p*dv_bi);
  h13=0.5#trace(p*dv_ai*p*dv_ci);
  h14=0.5#trace(p*dv_ai*p*dv_di);
  h15=0.5#trace(p*dv_ai*p*dv_e);
  h21=0.5#trace(p*dv_bi*p*dv_ai);
  h22=0.5#trace(p*dv_bi*p*dv_bi);
  h23=0.5#trace(p*dv_bi*p*dv_ci);
  h24=0.5#trace(p*dv_bi*p*dv_di);
  h25=0.5#trace(p*dv_bi*p*dv_e);
  h31=0.5#trace(p*dv_ci*p*dv_ai);
  h32=0.5#trace(p*dv_ci*p*dv_bi);
  h33=0.5#trace(p*dv_ci*p*dv_ci);
  h34=0.5#trace(p*dv_ci*p*dv_di);
  h35=0.5#trace(p*dv_ci*p*dv_e);
  h41=0.5#trace(p*dv_di*p*dv_ai);
  h42=0.5#trace(p*dv_di*p*dv_bi);
  h43=0.5#trace(p*dv_di*p*dv_ci);
  h44=0.5#trace(p*dv_di*p*dv_di);
  h45=0.5#trace(p*dv_di*p*dv_e);
  h51=0.5#trace(p*dv_e*p*dv_ai);
  h52=0.5#trace(p*dv_e*p*dv_bi);
  h53=0.5#trace(p*dv_e*p*dv_ci);
  h54=0.5#trace(p*dv_e*p*dv_di);
  h55=0.5#trace(p*dv_e*p*dv_e);  
  h = (h11 || h12 || h13 || h14 || h15) // 
      (h21 || h22 || h23 || h24 || h25) // 
      (h31 || h32 || h33 || h34 || h35) // 
      (h41 || h42 || h43 || h44 || h45) // 
      (h51 || h52 || h53 || h54 || h55);
      
  old_sigma = sigma_random // sigma_residual;
  sigma = old_sigma + inv(h)*score;
  sigma_random = sigma[1:4];
  sigma_residual = sigma[5];

  crit = max(abs((sigma-old_sigma)/old_sigma));
  beta_fixed = beta_fixed_new;
  beta_random = beta_random_new;
  niter = niter+1;

  print niter crit; 
 end;
 
 rss = (y - yhat)` * (y - yhat) / (nobs - 1);
 
 print "Converage after" niter "iterations"; 
 print crit beta_fixed sigma rss;
 

 print beta_random_matrix;
run;

In [ ]:
* IML given in class;

/* iml program for Mitscherlich model */
proc iml;
 use linplat;
 read all;
 /* specify initial values */
 aa=15;
 bb=0.33;
 cc=0.25;
 betaold=aa//bb//cc;
 yhat=aa*(1-bb*exp(-cc*x));
 print yhat;
 /* specify derivative matrix - analog of X matrix in linear model */
 xa=1-bb*exp(-cc*x);
 print xa;
 xb=-aa*exp(-cc*x);
 print xb;
 xc=aa#bb#x#exp(-cc*x);
 print xc;
 xx=xa||xb||xc;
 /* initialize YSTAR vector */
 ystar=y-yhat+xx*betaold;
 /* initialize convergence criterion and iteration counter */
 crit=1;
 niter=0;
start;
do while (crit>1e-08);
 niter=niter+1;
/* obtain new estimate of model parameters */
 beta=inv(xx`*xx)*xx`*ystar;
/* update parameter vector, YSTAR and derivative matrix */
 aa=beta[1,1]; 
 bb=beta[2,1]; 
 cc=beta[3,1]; yhat=aa*(1-bb*exp(-cc*x));
 *print yhat;
 xa=1-bb*exp(-cc*x);
 xb=-aa*exp(-cc*x);
 xc=aa#bb#x#exp(-cc*x);
 *print xa xb xc;
 xx=xa||xb||xc;
 ystar=y-yhat+xx*beta;
/* evaluate convergence status */
 crit=max(abs((beta-betaold)/betaold));
/* reset BETAOLD to allow convergence to be evaluated on next iteration */
 betaold=beta;
/* stop program from running forever if there is a mistake in the program or insufficient progress toward convergence */
 if niter>100 then goto terminate;
end;
goto success;
terminate:
  print "failed to converge in" niter "iterations";
  goto listing;
success:
  print "converged in" niter "iterations";
listing:
print niter crit aa bb cc;
finish;
run;


# Data Simulation

In [35]:
data simulated;
call streaminit(112999);
n_effects = 12;
n_sub = 20;

a = 10;
b = 30;
c = -5;
d = .5;
ai = 1;
bi = .5;
ci = .2;
di = 0.05;
var_ai = .677;
var_bi = 2.79;
var_ci = .172;
var_di = .769;
var_eu = 2;

do id=1 to n_effects;
 ai_x = ai + sqrt(var_ai)*rand("Normal");
 bi_x = bi + sqrt(var_bi)*rand("Normal");
 ci_x = ci + sqrt(var_ci)*rand("Normal");
 di_x = di + sqrt(var_di)*rand("Normal");
 
 do x=1 to n_sub;
  res = sqrt(var_eu)*rand("Normal");
  
   y = a + ai_x + (b + bi_x) / (1 + exp(- (c + ci_x + ((d + di_x) * (x))))) + res;
   cat = id*x;
   output;
 end;
end;

drop n_effects n_sub a b c d ai bi ci di ai_x bi_x ci_x di_x var_ai var_bi var_ci var_di var_eu res;


proc print data=simulated;
 where id=5;
run;

Obs,id,x,y,cat
81,5,1,12.7422,5
82,5,2,10.9221,10
83,5,3,16.9096,15
84,5,4,21.5711,20
85,5,5,32.9063,25
86,5,6,35.8726,30
87,5,7,36.9867,35
88,5,8,37.9947,40
89,5,9,40.3137,45
90,5,10,37.7359,50


In [36]:
goptions reset=all;
symbol i=join;
proc gplot data=simulated;
 plot y*x=id;
run;